In [2]:
import os
import os.path
import sys
from pathlib import Path
sys.path.insert(0, '..')
from elitetools import eddb
eddb.load_feeds()

In [11]:
origin_name="Dvorsi"
commodities = "Tobacco, Gold, Mineral Oil"
minimum_supply = 128
top_count = 5

In [12]:
commodity_names=[ c.strip() for c in commodities.split(',') ]
nearby_sources = eddb.commodity_sources_nearby(origin_name, commodity_names, minimum_supply, top_count)
nearby_sources

supply  buy_price  \
system      station                       commodity                         
Gubian      Ribeiro Industrial Silo       Mineral Oil      219        205   
Ling Deti   Young Station                 Gold             200      44541   
Maunggu     Sahni Agricultural Enterprise Mineral Oil     2722        105   
            van Vogt City                 Mineral Oil    88248        118   
Exioce      Blakytny Cultivation Hub      Mineral Oil      523        161   
                                          Tobacco          170       4296   
Bactondinks Comper Dock                   Gold             456      44346   
            Maughmer Platform             Gold             570      44345   
Gaula Wu    Friedman Works                Gold             597      48500   
Nzamnet     Draper Dock                   Gold             553      44277   
Negrito     Steakley Station              Mineral Oil  1423452        133   
                                          Tobacco       671754       4254   
Azrael      Rafferty's Mobius             Tobacco          161       4506   
Morana      Farrer Colony                 Tobacco          434       4309   
Aymifa      Polya Coliseum                Tobacco        71008       4352   

                                                      Pad Size  Sublight  \
system      station                       commodity                        
Gubian      Ribeiro Industrial Silo       Mineral Oil        S    1251.0   
Ling Deti   Young Station                 Gold               M    1521.0   
Maunggu     Sahni Agricultural Enterprise Mineral Oil        L      34.0   
            van Vogt City                 Mineral Oil        L     176.0   
Exioce      Blakytny Cultivation Hub      Mineral Oil        S    1604.0   
                                          Tobacco            S    1604.0   
Bactondinks Comper Dock                   Gold               M    8322.0   
            Maughmer Platform             Gold               M    8372.0   
Gaula Wu    Friedman Works                Gold               L      12.0   
Nzamnet     Draper Dock                   Gold               M     548.0   
Negrito     Steakley Station              Mineral Oil        L     131.0   
                                          Tobacco            L     131.0   
Azrael      Rafferty's Mobius             Tobacco            L     399.0   
Morana      Farrer Colony                 Tobacco            L     289.0   
Aymifa      Polya Coliseum                Tobacco            L     449.0   

                                                       distance  
system      station                       commodity              
Gubian      Ribeiro Industrial Silo       Mineral Oil       3.7  
Ling Deti   Young Station                 Gold              4.4  
Maunggu     Sahni Agricultural Enterprise Mineral Oil       9.3  
            van Vogt City                 Mineral Oil       9.3  
Exioce      Blakytny Cultivation Hub      Mineral Oil      10.2  
                                          Tobacco          10.2  
Bactondinks Comper Dock                   Gold             12.6  
            Maughmer Platform             Gold             12.6  
Gaula Wu    Friedman Works                Gold             12.6  
Nzamnet     Draper Dock                   Gold             13.4  
Negrito     Steakley Station              Mineral Oil      14.4  
                                          Tobacco          14.4  
Azrael      Rafferty's Mobius             Tobacco          14.5  
Morana      Farrer Colony                 Tobacco          15.5  
Aymifa      Polya Coliseum                Tobacco          17.7